In [ ]:
#| default_exp learner

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| export 
import math, torch, matplotlib.pyplot as plt
import fastcore.all as fc
from collections.abc import Mapping
from operator import attrgetter
from functools import partial
from copy import copy

from torch import optim
import torch.nn.functional as F

from miniai.conv import *

from fastprogress import progress_bar, master_bar

In [ ]:
import matplotlib as mpl
import torchvision.transforms.functional as TF
from contextlib import contextmanager
from torch import nn, tensor
from datasets import load_dataset, load_dataset_builder
from miniai.datasets import *
import logging
from fastcore.test import test_close

In [ ]:
torch.set_printoptions(precision=2, linewidth=160, sci_mode=False)
torch.manual_seed(1)
mpl.rcParams['image.cmap'] = 'gray'

In [ ]:
logging.disable(logging.WARNING)

## Learner

In [ ]:
x, y = 'image', 'label'
name = 'fashion_mnist'
dsd = load_dataset(name)

Extracting data files:   0%|          | 0/4 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/60000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Dataset fashion_mnist downloaded and prepared to /root/.cache/huggingface/datasets/fashion_mnist/fashion_mnist/1.0.0/8d6c32399aa01613d96e2cbc9b13638f359ef62bb33612b077b4c247f6ef99c1. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

Grab a single example from Dataset and check its shape. We will be working with flattened images, so we convert PIL to tensor and flatten it out to get 784 (28x28) long tensor

In [ ]:
ex = TF.to_tensor(dsd['train'][0][x])
ex.shape

torch.Size([1, 28, 28])

In [ ]:
torch.flatten(TF.to_tensor(dsd['train'][0][x])).shape

torch.Size([784])

In [ ]:
@inplace
def transformi(b):
    # b is a dictionary of image and label
    # import ipdb; ipdb.set_trace()
    b[x] = [torch.flatten(TF.to_tensor(o)) for o in b[x]]

In [ ]:
bs = 1024
tds = dsd.with_transform(transformi)

In [ ]:
# remove number of workers during debugging
dls = DataLoaders.from_dd(tds, bs, num_workers=4)

dt = dls.train
xb, yb = next(iter(dt))
xb.shape, yb[:10]

(torch.Size([1024, 784]), tensor([5, 4, 9, 4, 3, 0, 6, 5, 7, 6]))

Let's remind ourselves how `DataLoaders.from_dd` actually works. 

```python
def from_dd(cls, dd, batch_size, as_tuple=True, **kwargs):
        f = collate_dict(dd['train'])
        return cls(*get_dls(*dd.values(), bs=batch_size, collate_fn=f, **kwargs))
```
Let's first deal with the first line of code and `f` function. `f` is an internal collation function that itself returns a function returning collated `x` and `y` values from a given batch. This is how it is done:
1. `collate_dict` takes feature names from `dd['train']`: `image` and `label` and puts them into an itemgetter
2.  this itemgetter is applied to the result of a `default_collate` on a given Dataset returning a tuple of `x` and `y` values

```python
def collate_dict(ds):
    get = itemgetter(*ds.features) # get x and y values
    def _f(b): return get(default_collate(b))
    return _f
```

The second line of code (returning) can be broken down into several parts: call `get_dls` and wrap its return value into `cls`. Let's tackle it one by one.

```python
def get_dls(train_ds, valid_ds, bs, **kwargs):
    return (DataLoader(train_ds, bs, shuffle=True, **kwargs), DataLoader(valid_ds, bs*2, **kwargs))
```

`get_dls` takes a train and valid datasets and turns them into the respective DataLoaders (PyTorch). Our created `collate_dict` function (assigned to `f`) is passed to the DataLoader `__init__` method via `**kwargs`

Wrapping the resulting tuple of DataLoaders inside `cls` simply allows us to get them by calling .train and .vaild on our Dataloaders class.

```python
def __init__(self, *dls): self.train, self.valid = dls[:2]
```

Now we should fully understand the initial code above. We create dataloaders (train and valid), then we select train dataloader and get one batche of size 1024 from it.

```python
dls = DataLoaders.from_dd(tds, bs)
dt = dls.train
xb, yb = next(iter(dt))
```
During the call to iterator our collation function (`f`) comes in play and merges the 1024 element list of dictionaries into a tuple of `x` and `y` tensors.

Now let's move on to creating our first Learner framework.

##  Learner

First we create a simplified learner that still has some nice structure to it.

In [ ]:
class Learner:
    def __init__(self, model, dls, loss_func, lr, opt_func=optim.SGD): fc.store_attr()
    
    def one_batch(self):
        self.xb, self.yb = to_device(self.batch)
        self.preds = self.model(self.xb)
        self.loss = self.loss_func(self.preds, self.yb)
        if self.model.training:
            self.loss.backward()
            self.opt.step()
            self.opt.zero_grad()
        with torch.no_grad(): self.calc_stats()
        
    def calc_stats(self):
        acc = (self.preds.argmax(dim=-1)==self.yb).float().sum()
        self.accs.append(acc)
        n = len(self.xb)
        self.losses.append(self.loss * n)
        self.ns.append(n)
          
    def one_epoch(self, train):
        self.model.training = train
        dl = self.dls.train if train else self.dls.valid
        for _, self.batch in enumerate(dl): self.one_batch()
        n = sum(self.ns)
        print(self.epoch, self.model.training, 
              sum(self.losses).item()/n, sum(self.accs).item()/n)
        
    def fit(self, n_epochs):
        self.accs, self.losses, self.ns = [], [], []
        self.model.to(def_device)
        self.opt = self.opt_func(self.model.parameters(), self.lr)
        self.n_epochs = n_epochs
        for self.epoch in range(n_epochs):
            self.one_epoch(True)
            with torch.no_grad(): self.one_epoch(False)

In [ ]:
m, nh = 28*28, 50
model = nn.Sequential(nn.Linear(m, nh), nn.ReLU(), nn.Linear(nh, 10))

In [ ]:
learn = Learner(model, dls, F.cross_entropy, lr=0.2)
learn.fit(1)

0 True 1.1753048177083334 0.5986833333333333
0 False 1.1203291294642856 0.6135285714285714


## Basic Callbacks Learner 

Now let's improve greatly upon our previus iteration and add Callbacks to our Learner

In [ ]:
#|export
class CancelFitException(Exception): pass
class CancelBatchException(Exception): pass
class CancelEpochException(Exception): pass

In [ ]:
#|export
class Callback: order = 0

We use `getattr` to get a named attribute from an object; getattr(x, 'y') is equivalent to x.y.

In [ ]:
#|export
def run_cbs(cbs, method_nm, learn=None):
    for cb in sorted(cbs, key=attrgetter('order')):
        method = getattr(cb, method_nm, None)
        #?
        if method is not None: method(learn)

In [ ]:
class CompletionCB(Callback):
    def before_fit(self, learn): self.count = 0
    def after_batch(self, learn): self.count += 1
    def after_fit(self, learn): print(f'Completed {self.count} batches')

In [ ]:
cbs = [CompletionCB()]
run_cbs(cbs, 'before_fit')
run_cbs(cbs, 'after_batch')
run_cbs(cbs, 'after_fit')

Completed 1 batches


In [ ]:
class Learner:
    def __init__(self, model, dls, loss_func, lr, cbs, opt_func=optim.SGD): fc.store_attr()
    
    def one_batch(self):
        self.preds = self.model(self.batch[0])
        self.loss = self.loss_func(self.preds, self.batch[1])
        if self.model.training:
            self.loss.backward()
            self.opt.step()
            self.opt.zero_grad()
        
    def one_epoch(self, train: bool):
        self.model.train(train)
        self.dl = self.dls.train if train else self.dls.valid
        try:
            self.callback('before_epoch')
            for self.iter, self.batch in enumerate(self.dl):
                try:
                    self.callback('before_batch')
                    self.one_batch()
                    self.callback('after_batch')
                except CancelBatchException: pass
            # import ipdb; ipdb.set_trace()
            self.callback('after_epoch')
        except CancelEpochException: pass
            
    def fit(self, n_epochs):
        # import ipdb; ipdb.set_trace()
        self.n_epochs = n_epochs
        self.epochs = range(n_epochs)
        self.opt = self.opt_func(self.model.parameters(), self.lr)
        try:
            self.callback('before_fit')
            for self.epoch in self.epochs:
                self.one_epoch(True)
                self.one_epoch(False)
            self.callback('after_fit')
        except CancelFitException: pass
            
    def callback(self, method_nm): run_cbs(self.cbs, method_nm, self)

In [ ]:
m,nh = 28*28, 50
def get_model(): return nn.Sequential(nn.Linear(m, nh), nn.ReLU(), nn.Linear(nh, 10))

In [ ]:
model = get_model()
learn = Learner(model, dls, F.cross_entropy, lr=0.2, cbs=[CompletionCB()])
learn.fit(1)

Completed 59 batches


In [ ]:
tds

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 60000
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 10000
    })
})

```python
def get_dls(train_ds, valid_ds, bs, **kwargs):
    return (DataLoader(train_ds, bs, shuffle=True, **kwargs), DataLoader(valid_ds, bs*2, **kwargs))
```

In [ ]:
# 1024 * 2 because our dataloader for validation takes twice the batch size
math.ceil(60_000/1024) + math.ceil(10_000/(1024*2))

64

In [ ]:
#| export
class SingleBatchCB(Callback):
    order = 1
    def after_batch(self, learn): raise CancelFitException()

In [ ]:
learn = Learner(get_model(), dls, F.cross_entropy, lr=0.2, cbs=[SingleBatchCB(), CompletionCB()])
learn.fit(1)

## Metrics

We want to be able to use different metrics with our learner so we create a base class below.

In [ ]:
class Metric:
    def __init__(self): self.reset()
    
    def reset(self): self.vals, self.ns = [], []
    
    def add(self, inp, targ=None, n=1):
        self.last = self.calc(inp, targ)
        self.vals.append(self.last)
        self.ns.append(n)
    
    @property
    def value(self):
        ns = tensor(self.ns)
        # return weighted sum of metric
        return (tensor(self.vals)*ns).sum()/ns.sum()
        
    def calc(self, inps, targs): return inps

In [ ]:
class Accuracy(Metric):
    def calc(self, inps, targs): return (inps==targs).float().mean()

In [ ]:
acc = Accuracy()
acc.add(tensor([0,1,2,0,1,2]),tensor([0,1,1,2,1,0]))
acc.add(tensor([1,1,2,0,1]),tensor([0,1,1,2,1]))
acc.value

tensor(0.45)

Basic Metric can serve as a weighted average calculator because its `calc` method simply return input back.

In [ ]:
loss = Metric()
loss.add(0.6, n=32)
loss.add(0.9, n=2)
loss.value, round((0.6*32 + 0.9*2)/(32+2),2)

(tensor(0.62), 0.62)

## Some callbacks

Let's build upon the Metric class and add some metrics defined in PyTorch.

In [ ]:
#| export
from torcheval.metrics import MulticlassAccuracy, Mean

In [ ]:
metric = MulticlassAccuracy()
metric.update(tensor([0, 2, 1, 3]), tensor([0, 1, 2, 3]))
metric.compute()

tensor(0.50)

In [ ]:
metric.reset()
metric.compute()

tensor(nan)

Metrics are usually calculated using data that is on the `cpu`.

In [ ]:
#| export
def to_cpu(x):
    # import ipdb; ipdb.set_trace()
    """Recursively move data to cpu"""
    # mapping case
    if isinstance(x, Mapping): return {k: to_cpu(v) for k,v in x.items()}
    # list case
    if isinstance(x, list): return [to_cpu(o) for o in x]
    # tuple case (via list)
    if isinstance(x, tuple): return tuple(to_cpu(list(x)))
    # base case (recursive)
    res = x.detach().cpu()
    return res.float() if res.dtype == torch.float16 else res
                                          

In [ ]:
test = (tensor(1), tensor(2))

In [ ]:
to_cpu(test)

(tensor(1), tensor(2))

In [ ]:
#| export
class MetricsCB(Callback):
    def __init__(self, 
                 *ms, # list of metrics
                 **metrics, # dictionary of metrics
                ):
        # pool all unnamed metrics into a dictionary
        for o in ms: metrics[type(o).__name__] = o
        self.metrics = metrics
        self.all_metrics = copy(metrics)
        self.all_metrics['loss'] = self.loss = Mean()
        
    def _log(self, d): print(d)
    def before_fit(self, learn): learn.metrics = self
    def before_epoch(self, learn): [o.reset() for o in self.all_metrics.values()]
    
    # print metrics after each epoch
    def after_epoch(self, learn):
        # save log of all metrics
        log = {k: f'{v.compute():.3f}' for k,v in self.all_metrics.items()}
        log['epoch'] = learn.epoch
        log['train'] = 'train' if learn.model.training else 'eval'
        self._log(log)
    
    def after_batch(self,learn):
        # unpack all remaining values into *
        x,y,*_ = to_cpu(learn.batch)
        for m in self.metrics.values(): m.update(to_cpu(learn.preds), y)
        self.loss.update(to_cpu(learn.loss), weight=len(x))

In [ ]:
test = MetricsCB(1,'2',3.0,[4])
test.metrics

{'int': 1, 'str': '2', 'float': 3.0, 'list': [4]}

In [ ]:
def o(): return 1,2,3,4,5
x1, x2, *_ = o()

In [ ]:
#| export 
class DeviceCB(Callback):
    def __init__(self, device=def_device): fc.store_attr()
    def before_fit(self, learn):
        if hasattr(learn.model, 'to'): learn.model.to(self.device)
    def before_batch(self, learn): learn.batch = to_device(learn.batch, self.device)

In [ ]:
model = get_model()
metrics = MetricsCB(accuracy=MulticlassAccuracy())
learn = Learner(model, dls, F.cross_entropy, lr=0.2, cbs=[DeviceCB(), metrics])
learn.fit(1)

{'accuracy': '0.621', 'loss': '1.162', 'epoch': 0, 'train': 'train'}
{'accuracy': '0.676', 'loss': '0.879', 'epoch': 0, 'train': 'eval'}
